<h1>Golden Gate Cloning</h1>


In [ ]:
from opentrons import protocol_api
metadata = {
    "protocolName": "96 well plate golden gate assembly",
    "description": "Performs golden gate assembly in a 96 well plate for a predefined set of combinations of parts. You'll need to define your combinations in advance",
    "author": "JATD"
}

requirements = {
    "robotType": "Flex", # Ensure the robot type is Flex,
    "apiLevel": "2.23",
    }


def run(protocol: protocol_api.ProtocolContext):
    tips = protocol.load_labware("opentrons_flex_96_tiprack_50ul", "C2")
    tubes = protocol.load_labware("opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap", "D2")
    trash = protocol.load_trash_bin("A3")

    thermocycler = protocol.load_module("thermocycler", "A1")
    thermocycler_plate = thermocycler.load_labware("nest_96_wellplate_100ul_pcr_full_skirt")
    thermocycler.open_lid()

